In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aita-clean-dataset/aita_clean.csv


In [2]:
!pip install -qq bertopic


In [3]:
import pandas as pd
from bertopic import BERTopic

### Reading dataset and optionally sampling 

In [38]:
FILE_PATH = '/kaggle/input/aita-clean-dataset/aita_clean.csv'
df = pd.read_csv(FILE_PATH)
df = df.sample(3000, random_state=123)# Comment out this to disable sampling
df.head()

,id,timestamp,title,body,edited,verdict,score,num_comments,is_asshole
3425,90299e,1.531971e+09,WIBTA if i asked my boss about the money he sa...,I housesitted for my boss for a week while he ...,False,not the asshole,10,6.0,0
43612,ca5eg7,1.562495e+09,AITA for requesting my ex travel so I can see ...,This is a bit of a ridiculous situation going ...,False,no assholes here,9,18.0,0
50018,ciqpfg,1.564282e+09,AITA for calling someone on their bullshit,"I know the title sounds open and shut, but he...",1564284134.0,not the asshole,3,8.0,0
72920,dolsuc,1.572329e+09,AITA for wanting to sleep when my roommate stu...,I try to prioritize my sleep as a student and ...,False,not the asshole,9,17.0,0
37219,bydsp2,1.560035e+09,AITA for sleeping with my ex whilst seriously ...,We broke up a few months ago after 3 years. Si...,1560036097.0,asshole,3,9.0,1


#### Preprocesssing

In [39]:
# Concatenate the title and body columns
df['text'] = df['title'] + " " + df['body']


### Topic Modeling on Titles

In [41]:
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP

# Prepare embeddings
docs = df['title'].tolist()
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=True)




Batches:   0%|          | 0/94 [00:00<?, ?it/s]

In [42]:
# Train BERTopic
topic_model = BERTopic().fit(docs, embeddings)

# Run the visualization with the original embeddings
topic_model.visualize_documents(docs, embeddings=embeddings)

# # Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
# reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
# topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings)

In [43]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1002,-1_aita_for_my_to,"[aita, for, my, to, the, not, at, with, friend...",[AITA for being close friends with my half-sis...
1,0,464,0_wibta_if_my_to,"[wibta, if, my, to, told, tell, friend, the, d...",[WIBTA if I call my sister and her Fiancée out...
2,1,105,1_roommate_sleep_roommates_room,"[roommate, sleep, roommates, room, in, bed, ap...",[AITA for not wanting my friend’s friend to sl...
3,2,100,2_mother_mom_my_she,"[mother, mom, my, she, aita, for, in, mum, her...","[AITA for telling my(35f) mother to move out?,..."
4,3,69,3_wedding_ring_engagement_married,"[wedding, ring, engagement, married, not, my, ...",[AITA for refusing to go to my sisters wedding...
5,4,63,4_pay_back_for_to,"[pay, back, for, to, aita, credit, not, sellin...",[AITA for refusing to pay for a friends car i ...
6,5,62,5_aita_game_playing_what,"[aita, game, playing, what, election, directio...","[AITA for playing a board game on NYE, AITA fo..."
7,6,61,6_dog_dogs_puppy_neighbors,"[dog, dogs, puppy, neighbors, putting, aita, n...",[AITA for giving away my sister's dog without ...
8,7,56,7_girlfriend_gf_girl_her,"[girlfriend, gf, girl, her, she, with, that, t...","[AITA For Being Mad at My Girlfriend?, AITA fo..."
9,8,55,8_food_dinner_cooking_eat,"[food, dinner, cooking, eat, table, eating, pi...",[AITA for wanting to eat my food before everyo...


In [47]:
topic_model.get_topic_info()['Name']

0                           -1_aita_for_my_to
1                            0_wibta_if_my_to
2             1_roommate_sleep_roommates_room
3                         2_mother_mom_my_she
4           3_wedding_ring_engagement_married
5                           4_pay_back_for_to
6                    5_aita_game_playing_what
7                  6_dog_dogs_puppy_neighbors
8                    7_girlfriend_gf_girl_her
9                   8_food_dinner_cooking_eat
10                       9_dad_dads_father_my
11               10_yelling_at_blasting_music
12                 11_sister_sisters_my_being
13         12_school_graduation_student_class
14                      13_boyfriend_he_bf_go
15             14_girlfriend_gf_vacation_sick
16          15_laundry_cleaning_clean_washing
17             16_birthday_gift_party_present
18                17_quitting_job_work_notice
19                     18_car_bus_note_driver
20        19_family_familys_storming_vacation
21            20_christmas_gift_gi

### Topic Modeling on body

In [45]:
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP

# Prepare embeddings
docs = df['body'].tolist()
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=True)

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

In [46]:
# Train BERTopic
topic_model = BERTopic().fit(docs, embeddings)

# Run the visualization with the original embeddings
topic_model.visualize_documents(docs, embeddings=embeddings)

# # Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
# reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
# topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings)

TypeError: sequence item 728: expected str instance, float found

### Topic Modeling on title+body

In [ ]:
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP

# Prepare embeddings
docs = df['text'].tolist()
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=False)

In [ ]:
# Train BERTopic
topic_model = BERTopic().fit(docs, embeddings)

# Run the visualization with the original embeddings
topic_model.visualize_documents(docs, embeddings=embeddings)

# # Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
# reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
# topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings)